In [1]:
# %cd /content/drive/Shared drives/IML Project/Project

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf

from tensorflow.keras import Sequential

from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv3D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import PReLU
from tensorflow.keras.layers import MaxPool3D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import AveragePooling3D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

In [3]:
data_path = 'tf_datasets'
batch_size = 32

In [4]:
def decode(serialized_example):
    """
    Parses an image and label from the given `serialized_example`.
    It is used as a map function for `dataset.map`
    """

    features = tf.compat.v1.parse_single_example(
        serialized_example,
        features={
            'img_channels': tf.compat.v1.FixedLenFeature([], tf.int64),
            'img_height': tf.compat.v1.FixedLenFeature([], tf.int64),
            'img_width': tf.compat.v1.FixedLenFeature([], tf.int64),
            'img_raw': tf.compat.v1.FixedLenFeature([], tf.string),
            'sex': tf.compat.v1.FixedLenFeature([], tf.string),
            'age': tf.compat.v1.FixedLenFeature([], tf.float32),
            'label': tf.compat.v1.FixedLenFeature([], tf.string),
        })
    
    
    
    height = tf.cast(features['img_height'], tf.int32)
    width = tf.cast(features['img_width'], tf.int32)
    channels = tf.cast(features['img_channels'], tf.int32)
    image = tf.compat.v1.decode_raw(features['img_raw'], tf.float64)
    image = tf.reshape(image, (62, 96, 96, 1))
    label = tf.compat.v1.decode_raw(features['label'], tf.float64)
    sex = tf.cast(features['sex'], tf.string)
    age = tf.cast(features['age'], tf.float32)
    return image, label

In [5]:
train_dataset = tf.data.TFRecordDataset(os.path.join(data_path, 'train.tfrecords')).map(decode).batch(batch_size)
val_dataset = tf.data.TFRecordDataset(os.path.join(data_path, 'validation.tfrecords')).map(decode).batch(batch_size)
test_dataset = tf.data.TFRecordDataset(os.path.join(data_path, 'test.tfrecords')).map(decode).batch(batch_size)

In [6]:
def create_model():
    kernel_regularizer = tf.keras.regularizers.l2(0.001)
    he_normal = tf.keras.initializers.he_normal(seed=0)
    lecun_normal = tf.keras.initializers.lecun_normal(seed=0)

    input_3d = (62, 96, 96, 1)
    pool_3d = (2, 2, 2)

    model = Sequential()
    
    model.add(tf.keras.layers.InputLayer(input_shape=input_3d))
    model.add(Conv3D(filters=8,
                     kernel_size=3,
                     kernel_regularizer=kernel_regularizer,
                     kernel_initializer=he_normal))
    model.add(BatchNormalization())
    model.add(PReLU())
    model.add(MaxPool3D(pool_size=pool_3d, name='pool1'))

    model.add(Conv3D(filters=8,
                     kernel_size=3,
                     kernel_regularizer=kernel_regularizer,
                     kernel_initializer=he_normal))
    model.add(BatchNormalization())
    model.add(PReLU())
    model.add(MaxPool3D(pool_size=pool_3d, name='pool2'))

    model.add(Conv3D(filters=8,
                     kernel_size=3,
                     kernel_regularizer=kernel_regularizer,
                     kernel_initializer=he_normal))
    model.add(BatchNormalization())
    model.add(PReLU())
    model.add(MaxPool3D(pool_size=pool_3d, name='pool3'))

    model.add(Flatten())
    model.add(Dense(1024, name='dense1', kernel_regularizer=kernel_regularizer, kernel_initializer=he_normal))
    model.add(LeakyReLU())
    model.add(Dropout(0.5, name='dropout1'))

    model.add(Dense(512, activation='relu', name='dense2', kernel_regularizer=kernel_regularizer, kernel_initializer=he_normal))
    model.add(LeakyReLU())
    model.add(Dropout(0.5, name='dropout2'))

    model.add(Dense(3, activation='softmax', name='softmax', kernel_initializer=lecun_normal))

    model.compile(loss='categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(1e-5),
                  metrics=[tf.keras.metrics.CategoricalAccuracy()])

    return model

In [7]:
reducelr_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_categorical_accuracy',
                                                         factor=0.2,
                                                         patience=5,
                                                         min_delta=0.01,
                                                         verbose=1)
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(os.path.join('checkpoints', 'cp.ckpt'),
                                                         monitor='val_categorical_accuracy',
                                                         verbose=1,
                                                         save_best_only=True,
                                                         mode='max')

callbacks_list = [checkpoint_callback, reducelr_callback]

In [8]:
model = create_model()
model.summary()
# if os.path.exists(os.path.join('weights', 'best_weights.h5')):
#     model.load_weights(os.path.join('weights', 'best_weights.h5'))
#     print('Loaded weights')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 60, 94, 94, 8)     224       
_________________________________________________________________
batch_normalization (BatchNo (None, 60, 94, 94, 8)     32        
_________________________________________________________________
p_re_lu (PReLU)              (None, 60, 94, 94, 8)     4241280   
_________________________________________________________________
pool1 (MaxPooling3D)         (None, 30, 47, 47, 8)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 28, 45, 45, 8)     1736      
_________________________________________________________________
batch_normalization_1 (Batch (None, 28, 45, 45, 8)     32        
_________________________________________________________________
p_re_lu_1 (PReLU)            (None, 28, 45, 45, 8)     4

In [9]:
history = model.fit(train_dataset, validation_data=val_dataset, epochs=25, callbacks=callbacks_list, verbose=1)

Epoch 1/25
      3/Unknown - 154s 51s/step - loss: 6.7386 - categorical_accuracy: 0.1875WARNING:tensorflow:Can save best model only with val_categorical_accuracy available, skipping.
      3/Unknown - 154s 51s/step - loss: 6.7386 - categorical_accuracy: 0.1875

KeyboardInterrupt: 

In [14]:
%cp -r '/content/drive/Shared drives/IML Project/Project/img_array_mri.zip' '/content/drive/My Drive/Projects/Baseline-Alzheimers/images.zip'

cp: error reading '/content/drive/Shared drives/IML Project/Project/img_array_mri.zip': Input/output error
